### Caution of using modin.pandas:

before install modin by "pip install modin", install ray==1.1, aioredis==1.3.1 first to avoid errors

In [70]:
import json
import sys
import modin.pandas as modinpd
import pandas as pd
from perceval.backends.core.git import Git
from models.model_Commits import Commits

In [71]:

df = pd.read_csv("../gitrepository.csv")
github_source_url = df["Repos"].values
github_source_url = [
        'https://github.com/GeoNode/geonode',
#        'https://github.com/GeoNode/django-docker',
#        'https://github.com/GeoNode/geonode-client',
#        'https://github.com/GeoNode/django-osgeo-importer',
#        'https://github.com/qgis/QGIS',
#        'https://github.com/geoserver/geoserver'
]

In [72]:
df.head(10)

,Projects,Repos
0,GeoNode,https://github.com/GeoNode/django-osgeo-importer
1,GeoNode,https://github.com/GeoNode/django-docker
2,GeoNode,https://github.com/GeoNode/geonode-client
3,GeoNetwork,https://github.com/geonetwork/core-geonetwork
4,pycsw,https://github.com/geopython/pycsw
5,GeoNode,https://github.com/GeoNode/geonode
6,OSGeoLive,https://github.com/OSGeo/OSGeoLive
7,MapServer,https://github.com/mapserver/mapserver
8,QGIS,https://github.com/qgis/QGIS
9,deegree,https://github.com/deegree/deegree3


In [73]:
from perceval.backends.core.git import Git

# url for the git repo to analyze
for url in github_source_url:
    repo_url = url
    # directory for letting Perceval clone the git repo
    repo_dir = '/tmp/temp.git'
    counter = 0
    proj_name = str(df.loc[df["Repos"] == url.replace('/issues', '')]["Projects"].values[0]).lower()
    repo_name = repo_url.split('/')[-1]

    # create a Git object, pointing to repo_url, using repo_dir for cloning
    repo = Git(uri=repo_url, gitpath=repo_dir)
    repo_data = repo.fetch()
    
    #print(type(repo_data))
    # Fetch return a generator
    
    ####fw = open('./repo_fetch.json','w')
    ###fw.write('{\"all_commits\":[')
    
    all_commits = []

    # fetch all commits as an iterator, and iterate it printing each hash
    
    for commit in repo_data:
        counter += 1
        commit['data']['proj_id'] = proj_name
        commit['data']['repo'] = repo_name
        commit['data']['commit_id'] = proj_name+"#"+repo_name+"#"+str(counter)+"#"+commit['data']['commit']
        
        try:
            author= commit['data']['Author'].replace('<', ' ').replace('>', ' ').split()[0]
            author_email= commit['data']['Author'].replace('<', ' ').replace('>', ' ').split()[-1]
            commiter= commit['data']['Commit'].replace('<', ' ').replace('>', ' ').split()[0]
            commiter_email= commit['data']['Commit'].replace('<', ' ').replace('>', ' ').split()[-1]
            if author == author_email :
                author_email = ""
            if commiter == commiter_email:
                commiter_email = ""
            commit['data']['author_id'] = author
            commit['data']['author_email'] =  author_email
            commit['data']['commiter_id'] = commiter
            commit['data']['commiter_email'] = commiter_email
        except IndexError as e:
            print(commit['data']['Author'])
            print(commit['data']['Commit'])

        ###jschar_repo_data = json.dumps(commit['data'])
        all_commits.append(commit['data'])

        ###fw.write(jschar_repo_data+',')
    ###fw.write(']}')
    ####json.dump({'all_commits':all_commits}, fw)
    ####fw.close()
    print(counter)

    df_all_commits = pd.DataFrame(all_commits)
    #df_all_commits.head(10)

19271


In [167]:
#df_all_commits.describe()

,commit_SHA,commit_parents,commit_refs,author,author_timestamp,commiter,commit_timestamp,commit_message,files,proj_id,repo,commit_id,author_id,author_email,commiter_id,commiter_email,Merge,Signed-off-by
count,19271,19271,19271,19271,19271,19271,19271,19271,19271,19271,19271,19271,19271,19271,19271,19271,4787,1134
unique,19271,17618,262,423,18145,324,18867,16504,17239,1,1,19271,298,361,235,285,4774,6
top,a9e210d1485387a2e4cf938947fcb62e979bcd58,[4e64ebb98cc7f6e9dd5a2fb06427093181cd028b],[],Alessio Fabiani <alessio.fabiani@geo-solutions...,Wed Mar 17 10:00:52 2021 +0100,GitHub <noreply@github.com>,Tue Sep 1 10:17:40 2020 +1200,Merge branch 'master' of https://github.com/Ge...,"[{'file': 'requirements.txt', 'added': '1', 'r...",geonode,geonode,geonode#geonode#6b0d0a9ed3d0172c38e42481f66691...,Alessio,alessio.fabiani@geo-solutions.it,GitHub,noreply@github.com,04d2f5bf3 957ab13bf,[dependabot-preview[bot] <support@dependabot.c...
freq,1,14,19010,2499,4,4201,31,280,250,19271,19271,1,2501,4709,4201,4201,2,641


In [5]:
df_all_commits.columns

Index(['commit', 'parents', 'refs', 'Author', 'AuthorDate', 'Commit',
       'CommitDate', 'message', 'files', 'proj_id', 'repo', 'commit_id',
       'author_id', 'author_email', 'commiter_id', 'commiter_email', 'Merge',
       'Signed-off-by'],
      dtype='object')

In [74]:
df_all_commits.columns = ['commit_sha', 'commit_parents', 'commit_refs', 'author', 'author_timestamp', 'commiter',
       'commit_timestamp', 'commit_message', 'files', 'proj_id', 'repo', 'commit_id','author_aliase_id', 'author_email', 'commiter_aliase_id', 'commiter_email', 
       'Merge', 'Signed-off-by']

In [75]:
df_all_commits.head(5)

,commit_sha,commit_parents,commit_refs,author,author_timestamp,commiter,commit_timestamp,commit_message,files,proj_id,repo,commit_id,author_aliase_id,author_email,commiter_aliase_id,commiter_email,Merge,Signed-off-by
0,845d21bbd0d342b849e23e315a2b2208f7cd5abe,[],[],whit <whit@125c47a5-210e-49ec-a0c6-83efc728e0a1>,Tue Sep 29 21:07:38 2009 +0000,whit <whit@125c47a5-210e-49ec-a0c6-83efc728e0a1>,Tue Sep 29 21:07:38 2009 +0000,the trunk\n\ngit-svn-id: https://svn.opengeo.o...,[],geonode,geonode,geonode#geonode#1#845d21bbd0d342b849e23e315a2b...,whit,whit@125c47a5-210e-49ec-a0c6-83efc728e0a1,whit,whit@125c47a5-210e-49ec-a0c6-83efc728e0a1,NaN,NaN
1,a2345429e0a26a46c5e6c4de77f88ab15bbd118c,[845d21bbd0d342b849e23e315a2b2208f7cd5abe],[],whit <whit@125c47a5-210e-49ec-a0c6-83efc728e0a1>,Wed Sep 30 14:38:07 2009 +0000,whit <whit@125c47a5-210e-49ec-a0c6-83efc728e0a1>,Wed Sep 30 14:38:07 2009 +0000,Merge branch 'master' into local-svn\n\ngit-sv...,"[{'modes': ['000000', '100644'], 'indexes': ['...",geonode,geonode,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,whit,whit@125c47a5-210e-49ec-a0c6-83efc728e0a1,whit,whit@125c47a5-210e-49ec-a0c6-83efc728e0a1,NaN,NaN
2,ea3c56bc9969bda377bc060796f4d40fc794675b,[a2345429e0a26a46c5e6c4de77f88ab15bbd118c],[],whit <whit@125c47a5-210e-49ec-a0c6-83efc728e0a1>,Wed Sep 30 15:06:37 2009 +0000,whit <whit@125c47a5-210e-49ec-a0c6-83efc728e0a1>,Wed Sep 30 15:06:37 2009 +0000,Merge branch 'master' into local-svn\n\ngit-sv...,"[{'modes': ['100644', '100644'], 'indexes': ['...",geonode,geonode,geonode#geonode#3#ea3c56bc9969bda377bc060796f4...,whit,whit@125c47a5-210e-49ec-a0c6-83efc728e0a1,whit,whit@125c47a5-210e-49ec-a0c6-83efc728e0a1,NaN,NaN
3,7bae5032b5cc01e4dddf5e47917d069f2b9b3a82,[ea3c56bc9969bda377bc060796f4d40fc794675b],[],whit <whit@125c47a5-210e-49ec-a0c6-83efc728e0a1>,Wed Sep 30 15:08:46 2009 +0000,whit <whit@125c47a5-210e-49ec-a0c6-83efc728e0a1>,Wed Sep 30 15:08:46 2009 +0000,add activate\n\ngit-svn-id: https://svn.openge...,"[{'modes': ['100644', '100644'], 'indexes': ['...",geonode,geonode,geonode#geonode#4#7bae5032b5cc01e4dddf5e47917d...,whit,whit@125c47a5-210e-49ec-a0c6-83efc728e0a1,whit,whit@125c47a5-210e-49ec-a0c6-83efc728e0a1,NaN,NaN
4,b6b6de400838afbfc6ce46adf9c592f4ad3fa606,[7bae5032b5cc01e4dddf5e47917d069f2b9b3a82],[],whit <whit@125c47a5-210e-49ec-a0c6-83efc728e0a1>,Wed Sep 30 15:34:01 2009 +0000,whit <whit@125c47a5-210e-49ec-a0c6-83efc728e0a1>,Wed Sep 30 15:34:01 2009 +0000,"- make checkout conditional\n\n- new svn url, ...","[{'modes': ['100644', '100644'], 'indexes': ['...",geonode,geonode,geonode#geonode#5#b6b6de400838afbfc6ce46adf9c5...,whit,whit@125c47a5-210e-49ec-a0c6-83efc728e0a1,whit,whit@125c47a5-210e-49ec-a0c6-83efc728e0a1,NaN,NaN


In [76]:
df_all_commits['commit_timestamp'].apply(lambda x: pd.Timestamp(x))
df_all_commits['author_timestamp'].apply(lambda x: pd.Timestamp(x))

0        2009-09-29 21:07:38+00:00
1        2009-09-30 14:38:07+00:00
2        2009-09-30 15:06:37+00:00
3        2009-09-30 15:08:46+00:00
4        2009-09-30 15:34:01+00:00
                   ...            
19266    2021-10-04 00:01:11+00:00
19267    2021-10-04 00:01:15+00:00
19268    2021-10-04 00:01:18+00:00
19269    2021-10-04 00:01:24+00:00
19270    2021-10-04 00:01:29+00:00
Name: author_timestamp, Length: 19271, dtype: object

In [77]:
df_psql_commits = df_all_commits[['commit_id','proj_id','author_aliase_id', 'author_timestamp', 
'commiter_aliase_id', 'commit_timestamp', 'commit_message', 'commit_sha', 'commit_parents', 'commit_refs']].astype(str)

In [176]:
#df_psql_commits['commit_parents']

### For .astype(str)
list already turned into string

In [163]:
#df_psql_commits['commit_parents'].apply(lambda x: "".join([y+'#' for y in x]))

0                                                     [#]#
1        [#'#8#4#5#d#2#1#b#b#d#0#d#3#4#2#b#8#4#9#e#2#3#...
2        [#'#a#2#3#4#5#4#2#9#e#0#a#2#6#a#4#6#c#5#e#6#c#...
3        [#'#e#a#3#c#5#6#b#c#9#9#6#9#b#d#a#3#7#7#b#c#0#...
4        [#'#7#b#a#e#5#0#3#2#b#5#c#c#0#1#e#4#d#d#d#f#5#...
                               ...                        
19266    [#'#0#8#6#7#6#4#2#8#5#b#9#c#4#3#9#3#3#d#a#a#6#...
19267    [#'#0#8#6#7#6#4#2#8#5#b#9#c#4#3#9#3#3#d#a#a#6#...
19268    [#'#0#8#6#7#6#4#2#8#5#b#9#c#4#3#9#3#3#d#a#a#6#...
19269    [#'#0#8#6#7#6#4#2#8#5#b#9#c#4#3#9#3#3#d#a#a#6#...
19270    [#'#0#8#6#7#6#4#2#8#5#b#9#c#4#3#9#3#3#d#a#a#6#...
Name: commit_parents, Length: 19271, dtype: object

In [78]:
df_psql_commits.head(5)

,commit_id,proj_id,author_aliase_id,author_timestamp,commiter_aliase_id,commit_timestamp,commit_message,commit_sha,commit_parents,commit_refs
0,geonode#geonode#1#845d21bbd0d342b849e23e315a2b...,geonode,whit,Tue Sep 29 21:07:38 2009 +0000,whit,Tue Sep 29 21:07:38 2009 +0000,the trunk\n\ngit-svn-id: https://svn.opengeo.o...,845d21bbd0d342b849e23e315a2b2208f7cd5abe,[],[]
1,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,geonode,whit,Wed Sep 30 14:38:07 2009 +0000,whit,Wed Sep 30 14:38:07 2009 +0000,Merge branch 'master' into local-svn\n\ngit-sv...,a2345429e0a26a46c5e6c4de77f88ab15bbd118c,['845d21bbd0d342b849e23e315a2b2208f7cd5abe'],[]
2,geonode#geonode#3#ea3c56bc9969bda377bc060796f4...,geonode,whit,Wed Sep 30 15:06:37 2009 +0000,whit,Wed Sep 30 15:06:37 2009 +0000,Merge branch 'master' into local-svn\n\ngit-sv...,ea3c56bc9969bda377bc060796f4d40fc794675b,['a2345429e0a26a46c5e6c4de77f88ab15bbd118c'],[]
3,geonode#geonode#4#7bae5032b5cc01e4dddf5e47917d...,geonode,whit,Wed Sep 30 15:08:46 2009 +0000,whit,Wed Sep 30 15:08:46 2009 +0000,add activate\n\ngit-svn-id: https://svn.openge...,7bae5032b5cc01e4dddf5e47917d069f2b9b3a82,['ea3c56bc9969bda377bc060796f4d40fc794675b'],[]
4,geonode#geonode#5#b6b6de400838afbfc6ce46adf9c5...,geonode,whit,Wed Sep 30 15:34:01 2009 +0000,whit,Wed Sep 30 15:34:01 2009 +0000,"- make checkout conditional\n\n- new svn url, ...",b6b6de400838afbfc6ce46adf9c592f4ad3fa606,['7bae5032b5cc01e4dddf5e47917d069f2b9b3a82'],[]


In [79]:
df_psql_aliases = df_all_commits[['author_aliase_id', 'author_email']].rename(\
        columns={'author_aliase_id':'aliase_id', 'author_email': 'mailaddress'}).append( \
    df_all_commits[['commiter_aliase_id', 'commiter_email']].rename( \
        columns={'commiter_aliase_id':'aliase_id', 'commiter_email': 'mailaddress'}), ignore_index=True).drop_duplicates().astype(str)
df_psql_aliases['source'] = 'Github'

In [45]:
len(df_psql_aliases['aliase_id'])

420

In [80]:
df_psql_aliases.head(10)

,aliase_id,mailaddress,source
0,whit,whit@125c47a5-210e-49ec-a0c6-83efc728e0a1,Github
6,dwinslow,dwinslow@125c47a5-210e-49ec-a0c6-83efc728e0a1,Github
20,sbenthall,sbenthall@125c47a5-210e-49ec-a0c6-83efc728e0a1,Github
75,groldan,groldan@125c47a5-210e-49ec-a0c6-83efc728e0a1,Github
178,iwillig,iwillig@125c47a5-210e-49ec-a0c6-83efc728e0a1,Github
266,ahocevar,ahocevar@125c47a5-210e-49ec-a0c6-83efc728e0a1,Github
420,ltucker,ltucker@125c47a5-210e-49ec-a0c6-83efc728e0a1,Github
631,ingenieroariel,ingenieroariel@125c47a5-210e-49ec-a0c6-83efc72...,Github
670,David,dwinslow@opengeo.org,Github
676,Luke,ltucker@openplans.org,Github


In [81]:
all_filelogs = []
counter = 0
for filelog, commit_id, proj, repo in (df_all_commits[["files","commit_id","proj_id","repo"]].values):
    for log in filelog:
        counter+=1
        log['commit_id'] = commit_id
        log['id'] = proj+"#"+repo+"#"+str(counter)+"#"+log['file']
        #log['mode_before']= log['modes'][0]
        #log['mode_after']= log['modes'][-1]
    all_filelogs += filelog
    #print(filelog)
    #break
    
df_all_filelogs = pd.DataFrame(all_filelogs)
df_all_filelogs.head(10)

,modes,indexes,action,file,added,removed,commit_id,id,newfile
0,"[000000, 100644]","[000000000, bc76efc6b]",A,.gitignore,11,0,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,geonode#geonode#1#.gitignore,NaN
1,"[000000, 100644]","[000000000, b543a238a]",A,README.rst,15,0,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,geonode#geonode#2#README.rst,NaN
2,"[000000, 100644]","[000000000, c0ad9e067]",A,bootstrap.py,1246,0,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,geonode#geonode#3#bootstrap.py,NaN
3,"[000000, 100644]","[000000000, 505faa142]",A,docs/LICENSE.txt,1,0,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,geonode#geonode#4#docs/LICENSE.txt,NaN
4,"[000000, 100644]","[000000000, 5c2a33fa5]",A,pavement.py,136,0,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,geonode#geonode#5#pavement.py,NaN
5,"[000000, 100644]","[000000000, a8ed093de]",A,paver-minilib.zip,-,-,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,geonode#geonode#6#paver-minilib.zip,NaN
6,"[000000, 100644]","[000000000, 728bf0844]",A,setup.py,13,0,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,geonode#geonode#7#setup.py,NaN
7,"[000000, 100644]","[000000000, bbace3245]",A,shared/dev-paste.ini,65,0,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,geonode#geonode#8#shared/dev-paste.ini,NaN
8,"[000000, 100644]","[000000000, 7f48703b7]",A,shared/geonode-requirments.txt,19,0,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,geonode#geonode#9#shared/geonode-requirments.txt,NaN
9,"[000000, 100644]","[000000000, 0f45b989a]",A,src/.DS_Store,-,-,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,geonode#geonode#10#src/.DS_Store,NaN


In [82]:
df_all_filelogs.columns

Index(['modes', 'indexes', 'action', 'file', 'added', 'removed', 'commit_id',
       'id', 'newfile'],
      dtype='object')

In [83]:
df_all_filelogs.columns = ['modes', 'indexes', 'action', 'file_name', 'added', 'removed', 'commit_id',
       'filelog_id', 'newfile']

In [119]:
df_all_filelogs.describe()

,modes,indexes,action,file_name,added,removed,commit_SHA,filelog_id,newfile
count,89619,89619,89619,189304,187764,187764,189304,189304,6698
unique,25,66013,110,21853,1607,1449,19046,19046,6094
top,"[100644, 100644]","[000000000, e69de29bb]",M,requirements.txt,-,-,6c9cc752d8134944fde03491a1c60609ee518b0d,geonode#geonode#12846#6c9cc752d8134944fde03491...,geonode/client/templates/geoext/maps/map_geoex...
freq,51798,193,47778,2560,49087,49087,3180,3180,3


In [120]:
df_all_filelogs['action'].unique()

array(['A', 'M', 'D', 'R100', nan, 'R093', 'R092', 'R097', 'C074', 'R095',
       'R098', 'R099', 'R096', 'R091', 'R082', 'R069', 'R072', 'R066',
       'R050', 'R073', 'R068', 'R075', 'MM', 'MA', 'AA', 'R058', 'R077',
       'R054', 'R087', 'R094', 'R081', 'R076', 'RM', 'MR', 'R053', 'C061',
       'R065', 'R067', 'R052', 'R055', 'R051', 'AM', 'AR', 'R057', 'R084',
       'R089', 'R059', 'R083', 'RR', 'R088', 'RA', 'C097', 'C096', 'C100',
       'R061', 'R085', 'C090', 'R062', 'R074', 'R070', 'C098', 'C086',
       'DD', 'C073', 'C066', 'C092', 'R071', 'R060', 'C099', 'C067',
       'C064', 'R080', 'C055', 'C054', 'R086', 'CM', 'C091', 'C051',
       'C088', 'C095', 'C089', 'R064', 'C065', 'C057', 'C050', 'C058',
       'R079', 'C083', 'C069', 'C072', 'C079', 'R063', 'C059', 'C052',
       'C060', 'AC', 'C080', 'C063', 'C053', 'C084', 'C094', 'R056',
       'R078', 'C082', 'C070', 'C056', 'C093', 'R090', 'C081', 'C077',
       'C076'], dtype=object)

## Note: Explaination of Commit Information

1. Parents mean what this commit is based on, it's the SHA of another commit
2. In files, modes means file mod in linux system, index change everytime
3. added, removed means lines being modified


In [84]:
df_psql_filelogs = df_all_filelogs[['filelog_id', 'commit_id', 'modes', 'indexes', 'action', 'file_name', 'added', 'removed', 'newfile']].astype(str)

In [85]:
#df_psql_filelogs['mode_before']= df_psql_filelogs['modes'].apply(lambda x: x[0])

In [86]:
df_psql_filelogs.head(10)

,filelog_id,commit_id,modes,indexes,action,file_name,added,removed,newfile
0,geonode#geonode#1#.gitignore,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,"['000000', '100644']","['000000000', 'bc76efc6b']",A,.gitignore,11,0,nan
1,geonode#geonode#2#README.rst,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,"['000000', '100644']","['000000000', 'b543a238a']",A,README.rst,15,0,nan
2,geonode#geonode#3#bootstrap.py,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,"['000000', '100644']","['000000000', 'c0ad9e067']",A,bootstrap.py,1246,0,nan
3,geonode#geonode#4#docs/LICENSE.txt,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,"['000000', '100644']","['000000000', '505faa142']",A,docs/LICENSE.txt,1,0,nan
4,geonode#geonode#5#pavement.py,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,"['000000', '100644']","['000000000', '5c2a33fa5']",A,pavement.py,136,0,nan
5,geonode#geonode#6#paver-minilib.zip,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,"['000000', '100644']","['000000000', 'a8ed093de']",A,paver-minilib.zip,-,-,nan
6,geonode#geonode#7#setup.py,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,"['000000', '100644']","['000000000', '728bf0844']",A,setup.py,13,0,nan
7,geonode#geonode#8#shared/dev-paste.ini,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,"['000000', '100644']","['000000000', 'bbace3245']",A,shared/dev-paste.ini,65,0,nan
8,geonode#geonode#9#shared/geonode-requirments.txt,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,"['000000', '100644']","['000000000', '7f48703b7']",A,shared/geonode-requirments.txt,19,0,nan
9,geonode#geonode#10#src/.DS_Store,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,"['000000', '100644']","['000000000', '0f45b989a']",A,src/.DS_Store,-,-,nan


## Attention!
modin.pandas does not support to_sql!!!!!

In [87]:
from sqlalchemy import create_engine
from config.database import HOST, PORT, USER, PASSWORD, DATABASE

In [91]:
psql_engine = create_engine("postgresql://"+USER+":"+PASSWORD+"@"+HOST+":"+str(PORT)+"/"+DATABASE)

In [92]:
from sqlalchemy import types as sqltype
df_psql_filelogs.to_sql(name='filelog', con = psql_engine, if_exists= 'append', index= False)#,\
                    #dtype = {"modes": sqltype.Text,"indexes": sqltype.Text,"action":sqltype.Text,"file": sqltype.Text,"added":sqltype.Text,"removed":sqltype.Text,"newfile":sqltype.Text})

# multi is slower

In [94]:
df_psql_filelogs_result = pd.read_sql('filelog', psql_engine)
df_psql_filelogs_result.head(10)

,filelog_id,commit_id,modes,indexes,action,file_name,added,removed,newfile
0,geonode#geonode#1#.gitignore,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,"['000000', '100644']","['000000000', 'bc76efc6b']",A,.gitignore,11,0,nan
1,geonode#geonode#2#README.rst,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,"['000000', '100644']","['000000000', 'b543a238a']",A,README.rst,15,0,nan
2,geonode#geonode#3#bootstrap.py,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,"['000000', '100644']","['000000000', 'c0ad9e067']",A,bootstrap.py,1246,0,nan
3,geonode#geonode#4#docs/LICENSE.txt,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,"['000000', '100644']","['000000000', '505faa142']",A,docs/LICENSE.txt,1,0,nan
4,geonode#geonode#5#pavement.py,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,"['000000', '100644']","['000000000', '5c2a33fa5']",A,pavement.py,136,0,nan
5,geonode#geonode#6#paver-minilib.zip,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,"['000000', '100644']","['000000000', 'a8ed093de']",A,paver-minilib.zip,-,-,nan
6,geonode#geonode#7#setup.py,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,"['000000', '100644']","['000000000', '728bf0844']",A,setup.py,13,0,nan
7,geonode#geonode#8#shared/dev-paste.ini,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,"['000000', '100644']","['000000000', 'bbace3245']",A,shared/dev-paste.ini,65,0,nan
8,geonode#geonode#9#shared/geonode-requirments.txt,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,"['000000', '100644']","['000000000', '7f48703b7']",A,shared/geonode-requirments.txt,19,0,nan
9,geonode#geonode#10#src/.DS_Store,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,"['000000', '100644']","['000000000', '0f45b989a']",A,src/.DS_Store,-,-,nan


In [95]:
import psycopg2

def add_aliase(aliase_id, mailaddress, source):

    try:
        db = psycopg2.connect(
            host=HOST,
            port=PORT,
            user=USER,
            password=PASSWORD,
            database=DATABASE)
            #charset='utf8')
        cursor = db.cursor()
    except Exception as e:
        logging.error("Database connect error:%s" % e)

    #aliase_id, mailaddress = aliase.replace('<', ' ').replace('>', ' ').split()
    sql_aliase = """INSERT INTO aliase(aliase_id, mailaddress, source)
                                        values('%s', '%s', '%s')""" % (aliase_id, mailaddress, source)
    try:
        db.commit()
        cursor.execute(sql_aliase)
        db.commit()
    except Exception as err:
        sql_aliase = """UPDATE aliase SET mailaddress='%s', source='%s' WHERE aliase_id='%s' """ % (mailaddress, source, aliase_id)
        #print(err)
        pass
    return aliase_id

In [96]:
#df_psql_aliases.to_sql(name='aliase', con = psql_engine, if_exists= 'append', index= False, method= 'multi')


for aliase_id, mailaddress, source in df_psql_aliases.values :
    add_aliase(aliase_id, mailaddress, source)

In [97]:
df_psql_commits.to_sql(name='commit', con = psql_engine, if_exists= 'append', index= False)

In [98]:
df_psql_commit_result = pd.read_sql('commit', psql_engine)
df_psql_commit_result.head(10)

,commit_id,proj_id,commiter_aliase_id,thread_ids,commit_timestamp,commit_message,author_aliase_id,commit_sha,author_timestamp,commit_parents,commit_refs
0,geonode#geonode#1#845d21bbd0d342b849e23e315a2b...,geonode,whit,None,2009-09-29 21:07:38,the trunk\n\ngit-svn-id: https://svn.opengeo.o...,whit,845d21bbd0d342b849e23e315a2b2208f7cd5abe,2009-09-29 21:07:38,[],[]
1,geonode#geonode#2#a2345429e0a26a46c5e6c4de77f8...,geonode,whit,None,2009-09-30 14:38:07,Merge branch 'master' into local-svn\n\ngit-sv...,whit,a2345429e0a26a46c5e6c4de77f88ab15bbd118c,2009-09-30 14:38:07,['845d21bbd0d342b849e23e315a2b2208f7cd5abe'],[]
2,geonode#geonode#3#ea3c56bc9969bda377bc060796f4...,geonode,whit,None,2009-09-30 15:06:37,Merge branch 'master' into local-svn\n\ngit-sv...,whit,ea3c56bc9969bda377bc060796f4d40fc794675b,2009-09-30 15:06:37,['a2345429e0a26a46c5e6c4de77f88ab15bbd118c'],[]
3,geonode#geonode#4#7bae5032b5cc01e4dddf5e47917d...,geonode,whit,None,2009-09-30 15:08:46,add activate\n\ngit-svn-id: https://svn.openge...,whit,7bae5032b5cc01e4dddf5e47917d069f2b9b3a82,2009-09-30 15:08:46,['ea3c56bc9969bda377bc060796f4d40fc794675b'],[]
4,geonode#geonode#5#b6b6de400838afbfc6ce46adf9c5...,geonode,whit,None,2009-09-30 15:34:01,"- make checkout conditional\n\n- new svn url, ...",whit,b6b6de400838afbfc6ce46adf9c592f4ad3fa606,2009-09-30 15:34:01,['7bae5032b5cc01e4dddf5e47917d069f2b9b3a82'],[]
5,geonode#geonode#6#06a761161fc10b61ee87ac50497c...,geonode,whit,None,2009-09-30 15:54:03,add another mvn install\n\ngit-svn-id: https:/...,whit,06a761161fc10b61ee87ac50497c31b0918f3817,2009-09-30 15:54:03,['b6b6de400838afbfc6ce46adf9c592f4ad3fa606'],[]
6,geonode#geonode#7#49aa25f063e7ea11846c915a4d71...,geonode,dwinslow,None,2009-09-30 16:40:22,"Add dummied out rest service in GeoServer, set...",dwinslow,49aa25f063e7ea11846c915a4d715ec8a209a4d9,2009-09-30 16:40:22,['06a761161fc10b61ee87ac50497c31b0918f3817'],[]
7,geonode#geonode#8#15871f961475fd31d007a516ebaa...,geonode,whit,None,2009-09-30 17:11:23,silly merge\n\ngit-svn-id: https://svn.opengeo...,whit,15871f961475fd31d007a516ebaaf9df17c95c3b,2009-09-30 17:11:23,['49aa25f063e7ea11846c915a4d715ec8a209a4d9'],[]
8,geonode#geonode#9#f6ae1033b6b1dabb381afa92cd93...,geonode,whit,None,2009-09-30 17:13:21,fix cmd order\n\ngit-svn-id: https://svn.openg...,whit,f6ae1033b6b1dabb381afa92cd938187dc2c2b55,2009-09-30 17:13:21,['15871f961475fd31d007a516ebaaf9df17c95c3b'],[]
9,geonode#geonode#10#91768613a001af037a4a0fbbc67...,geonode,whit,None,2009-09-30 17:35:21,"adding a ""future"" reqs file\n\ngit-svn-id: htt...",whit,91768613a001af037a4a0fbbc678d7582a124971,2009-09-30 17:35:21,['f6ae1033b6b1dabb381afa92cd938187dc2c2b55'],[]


In [184]:
df_psql_test = pd.DataFrame({'balance': [10000.000], 'account':
                   ['13579.SH'], 'charges': [0.0005]})
df_psql_test.to_sql(name='balance', con=psql_engine, if_exists='append',
            index=False, index_label='id')

TypeError: can't pickle sqlalchemy.cprocessors.UnicodeResultProcessor objects